In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [7]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [8]:
import numpy as np

In [9]:
train_data, test_data = imdb['train'], imdb['test']

In [10]:
training_sentences = []
training_labels = []

In [11]:
testing_sentences = []
testing_labels = []

In [12]:
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

In [13]:
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

In [14]:
training_labels_final = np.array(training_labels)

In [15]:
testing_labels_final =  np.array(testing_labels)

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
t = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [22]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [25]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [30]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4819 - accuracy: 0.7571 - val_loss: 0.3487 - val_accuracy: 0.8464
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2316 - accuracy: 0.9101 - val_loss: 0.3713 - val_accuracy: 0.8388
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0841 - accuracy: 0.9782 - val_loss: 0.4603 - val_accuracy: 0.8281
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0198 - accuracy: 0.9975 - val_loss: 0.5364 - val_accuracy: 0.8259
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0048 - accuracy: 0.9998 - val_loss: 0.5998 - val_accuracy: 0.8270
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6492 - val_accuracy: 0.8281
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.1209e-04 - accuracy: 1.0000 - val_loss: 0.6913 - val_accuracy: 0.8290
Ep

In [32]:
e = model.layers[0]
weights = e.get_weights()[0]
weights.shape

(10000, 16)

In [33]:
import io

out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()